In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import nltk
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest, chi2
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix

In [ ]:
hate = pd.read_csv("/Volumes/DATA/UNIMI/Text Mining and Sentiment Analysis/labeled.csv", sep=',')

In [ ]:
hate.head(5)

In [ ]:
drop_cols = ["count", "hate_speech", "offensive_language", "neither", "Unnamed: 0"]
df =hate.drop(drop_cols, axis = 1)

In [ ]:
df

In [ ]:
def remove_punc(corpus):
    punt = list(string.punctuation)
    cleaned_tokens = []
    for w in nltk.word_tokenize(corpus):
        w = w.lower()
        if w not in punt:
            cleaned_tokens.append(w)
    cleaned_text = " ".join(cleaned_tokens)
    return cleaned_text

In [ ]:
from sklearn.model_selection import train_test_split
X = df['tweet']
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
X_test.shape

In [ ]:
#Plotting histogram of labels to see how they are distributed
y.plot.hist(bins=5, rwidth=2,
                   color='#607c8e')
plt.title('Histogram label')
plt.xlabel('Labels')
plt.ylabel('Counts')

In [ ]:
class PreProcess():

    def _init_(self):
        return

    def fit( self, X, y = None ):
        return self

    def clean (self, x):
        cleaned_text = remove_punc(x)
        return cleaned_text

    def transform(self, X, y = None):
        return X.apply(self.clean)

In [ ]:
pipe_clean = Pipeline([('clean', PreProcess()), 
                       ('tfidf', TfidfVectorizer())])

In [ ]:
gbdt = Pipeline([('pre_process', pipe_clean),
                    ('select', SelectKBest(score_func = chi2, k = 1000)),
                     ("std_scaler", StandardScaler(with_mean = False)),
                    ('classify', GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0))])
gbdt.fit(X_train, y_train)

In [ ]:
y_pred = gbdt.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average = 'micro')

In [ ]:
gbdt.predict(pd.Series('''As a woman you shouldn't complain about cleaning up your house'''))

In [ ]:
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])
sn.heatmap(confusion_matrix, annot=True)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
hate2 = pd.read_csv("/Users/namnguyen/Downloads/hatespeech-master/data.csv", sep=',', encoding = "utf-8")

In [ ]:
hate3 = pd.read_csv("/Users/namnguyen/Downloads/hatespeech-master/NAACL_SRW_2016.csv", sep=',')

In [ ]:
hate2.shape

In [ ]:
hate3

In [ ]:
type(hate3['tid'][0])

In [ ]:
hate2

In [ ]:
[hate2['tid'] == "#MKR"]
i = 0
for x in hate2['tid']:
    if x == "#MKR":
        i += 1
print(i)

In [ ]:
hate2['tid'] = hate2['tid'].astype(int)

In [ ]:
type(hate2['tid'][0])

In [ ]:
hate3 = hate3.drop_duplicates(subset='tid', keep="last")

In [ ]:
hate3

In [ ]:
hate2 = hate2.drop_duplicates(subset='tid', keep="last")

In [ ]:
hate2

In [ ]:
hate2['text'].isnull().values.any()

In [ ]:
hate_df = pd.merge(hate2, hate3, how='inner', on=['tid'])

In [ ]:
hate_df

In [ ]:
hate_df = hate_df.dropna(axis='rows')

In [ ]:
hate_df

In [ ]:
type_encode = []
for x in hate_df['type']:
    if x == "racism":
        type_encode.append(1)
    if x == "sexism":
        type_encode.append(2)
    if x == "none":
        type_encode.append(0)

In [ ]:
len(type_encode)

In [ ]:
hate_df['type'] = type_encode

In [ ]:
hate_df

In [ ]:
from sklearn.model_selection import train_test_split
X = hate_df['text']
y = hate_df['type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
#Plotting histogram of labels to see how they are distributed
y.plot.hist(bins=5, rwidth=4,
                   color='#607c8e')
plt.title('Histogram label')
plt.xlabel('Labels')
plt.ylabel('Counts')
plt.show()

In [ ]:
hate_df['type'].value_counts()

In [ ]:
hate_data2_model = gbdt.fit(X_train, y_train)

In [ ]:
y_pred = hate_data2_model.predict(X_test)

In [ ]:
f1_score(y_test, y_pred, average = 'micro')

In [ ]:
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])
sn.heatmap(confusion_matrix, annot=True)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

## Neural Network method

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(42)

In [ ]:
chars = string.ascii_lowercase + string.whitespace + '#'
CHAR_INDEX = dict((c, i) for i, c in enumerate(chars))

In [ ]:
def preprocess(s):
    out = ""
    for c in s.lower():
        if c in CHAR_INDEX.keys():
            out += c
        else:
            out += '#'
    return "".join(out)

In [ ]:
def string_to_matrix(s, n=2):
    z = preprocess(s)
    C = np.zeros((len(CHAR_INDEX), len(CHAR_INDEX)))
    for a, b in nltk.ngrams(z, n=n):
        C[CHAR_INDEX[a], CHAR_INDEX[b]] += 1
    C /= C.max()
    return C

In [ ]:
preprocess(X[1])

In [ ]:
X

In [ ]:
string_to_matrix(X[1]).shape

In [ ]:
N_LABELS = y.nunique()
V = len(CHAR_INDEX) * len(CHAR_INDEX)

In [ ]:
class Simple2Gram(nn.Module):
    
    def __init__(self, num_labels, size):
        super(Simple2Gram, self).__init__()
        self.linear = nn.Linear(size, num_labels)
    
    def forward(self, vec):
        return F.softmax(self.linear(vec), dim=1)
        #return F.log_softmax(self.linear(vec), dim=1)

In [ ]:
def vector(s, n=2):
    vec = torch.tensor(string_to_matrix(s, n=n)).float()
    return vec.view(1, -1)

def target(label):
    return torch.LongTensor([label])

In [ ]:
model = Simple2Gram(N_LABELS, V)

In [ ]:
with torch.no_grad():
    sample = X[0]
    vec = vector(sample, n=2)
    log_probs = model(vec)

In [ ]:
N_LABELS

In [ ]:
log_probs

In [ ]:
np.argmax(log_probs.numpy())

In [ ]:
#Training
loss = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
len(X)

In [ ]:
for epoch in range(50):
    for sample in range(len(X_train)):
        model.zero_grad()
        vec = vector(X_train.iloc[sample])
        tar = target(y_train.iloc[sample])
        log_probs = model(vec)
        print("log prob", log_probs)
        print('tar', tar)
        L = loss(log_probs, tar)
        L.backward()
        optimizer.step()

In [ ]:
from torch.distributions import Categorical

In [ ]:
entropy = lambda x: Categorical(probs=x).entropy()

In [ ]:
predictions, y_true = [], []
with torch.no_grad():
    for sample in range(len(y_test)):
        vec = vector(X_test.iloc[sample], n=2)
        predictions.append(model(vec))
        y_true.append(y_test.iloc[sample])
y_true = np.array(y_true)

In [ ]:
E = sum([entropy(p) for p in predictions])
E

In [ ]:
y_pred = [np.argmax(p.numpy()) for p in predictions]
y_pred

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_true, y_pred))

In [ ]:
f1_score(y_test, y_pred, average = 'micro')

In [ ]:
#from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
#Plotting histogram of labels to see how they are distributed
y_test.plot.hist(bins=5, rwidth=4,
                   color='#607c8e')
plt.title('Histogram label')
plt.xlabel('Labels')
plt.ylabel('Counts')
plt.show()

In [ ]:
y_pred.count(1)

In [ ]:
y_pred.count(2)

In [ ]:
y_pred.count(0)

In [ ]:
hate_df

In [ ]:
hate_df['tweet'] = hate_df['text']

In [ ]:
hate_df

In [ ]:
hate_df = hate_df.drop(['tid', 'text', 'date;'], axis = 1)

In [ ]:
hate_df

In [ ]:
hate_df.columns = ['class', 'tweet']

In [ ]:
hate_df

In [ ]:
hate_df = hate_df.append(df)

In [ ]:
hate_df

In [ ]:
#Plotting histogram of labels to see how they are distributed
hate_df['class'].plot.hist(bins=5, rwidth=2,
                   color='#607c8e')
plt.title('Histogram label')
plt.xlabel('Labels')
plt.ylabel('Counts')

In [ ]:
X = hate_df['tweet']
y = hate_df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
for epoch in range(50):
    for sample in range(len(X_train)):
        model.zero_grad()
        vec = vector(X_train.iloc[sample])
        tar = target(y_train.iloc[sample])
        log_probs = model(vec)
        print("log prob", log_probs)
        print('tar', tar)
        L = loss(log_probs, tar)
        L.backward()
        optimizer.step()

In [ ]:
predictions, y_true = [], []
with torch.no_grad():
    for sample in range(len(y_test)):
        vec = vector(X_test.iloc[sample], n=2)
        predictions.append(model(vec))
        y_true.append(y_test.iloc[sample])
y_true = np.array(y_true)

In [ ]:
y_pred = [np.argmax(p.numpy()) for p in predictions]

In [ ]:
print(classification_report(y_true, y_pred))

In [ ]:
f1_score(y_test, y_pred, average = 'micro')

In [ ]:
confusion_matrix(y_test, y_pred)